# Versión dos del modelo
En "maria_modelv1.ipynb" he creado un documento sqlite con el dataframe de de los datos de la clasificación de cada equipo (ejercicio 10). Voy a intentar crear el modelo de la versión v1 utilizando los dos data frames (laliga.sqlite, clasification.sqlite) sin hacerles un merge porque me falla la RAM.
 
Update1: Tengo que hacerles un merge a los dataframes porque tienen que tener las mismas dimesiones.

Update2: LA kernel muere al juntar los dos dataframes. (En la versión 1, para una season iba ok, pero es demasiado poco).

Update3: Ahora funciona, pero el el dataframe de train solo tiene una season. Con 3 seasons también, pero me ha tardado 20 min en hacer el train, casi me muero.

Update4: Has tocado los dafatframe df_train y df_test. El test está bien, me faltaban condiciones antes, el otro está mal. Hay que intentar unirlo de otra manera para hacer bien el train.


In [41]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier # el modelo
from sklearn.metrics import accuracy_score, classification_report # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int

In [42]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [43]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [44]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [45]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None
#estas dos columnas nos harían falta
df_games["goals_home"] = df_games["score"].str.split(":").str[0]
df_games["goals_visitor"] = df_games["score"].str.split(":").str[1]

df_games.loc[df["goals_home"] > df_games["goals_visitor"], "result"] = 'home'
df_games.loc[df["goals_home"] == df_games["goals_visitor"], "result"] = 'tie'
df_games.loc[df["goals_home"] < df_games["goals_visitor"], "result"] = 'visitor'

In [46]:
df_games.drop(["goals_home", "goals_visitor", "time", "date", "score"], axis=1, inplace=True)  
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,visitor
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,visitor
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


# Empieza el modelo (RandomForestClassifier)
Nuestro modelo/problema es de tipo clasificación. La predicción nos tiene que dar uno de los tres valores, NO es una variable continua.

Ya tenemos los datos. Ahora hay que definir nuestras variables. X son los datos que vamos a tener en cuenta, e Y es la "variable etiqueta", es decir, la que tenemos que predecir. La etiqueta tiene que ser de tipo int, empezamos con eso.

In [47]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "2000:2010"


In [48]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train.loc[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1]) & 
                     (df_teams_train["matchday"] == matchday) & (df_teams_train["division"] == division)]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
20248,2009-2010,1,1.0,Real Madrid,3,11.0,2.0,9.0,3.0,0.0,0.0,9.0
20249,2009-2010,1,2.0,Barcelona,3,10.0,2.0,8.0,3.0,0.0,0.0,9.0
20250,2009-2010,1,3.0,Athletic,3,5.0,2.0,3.0,3.0,0.0,0.0,9.0
20251,2009-2010,1,4.0,RCD Mallorca,3,7.0,1.0,6.0,2.0,0.0,1.0,7.0
20252,2009-2010,1,5.0,Valencia,3,8.0,4.0,4.0,2.0,0.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35419,2000-2001,1,16.0,Real Sociedad,3,4.0,7.0,-3.0,0.0,1.0,2.0,2.0
35420,2000-2001,1,17.0,UD Las Palmas,3,3.0,6.0,-3.0,0.0,1.0,2.0,2.0
35421,2000-2001,1,18.0,Real Oviedo,3,2.0,4.0,-2.0,0.0,2.0,1.0,1.0
35422,2000-2001,1,19.0,Racing,3,3.0,8.0,-5.0,0.0,2.0,1.0,1.0


In [49]:
df_games_train = df_games
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_train["season"] = df_games_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_train = df_games_train[(df_games_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_train["season"].str.split("-").str[1] <= season_train.split(":")[1]) 
                    & (df_games_train["division"] == division) & (df_games_train["matchday"] == matchday)]
df_games_train


,season,division,matchday,home_team,away_team,result
17858,2000-2001,1,3,Barcelona,Racing,home
17859,2000-2001,1,3,Real Madrid,Athletic,home
17860,2000-2001,1,3,Real Zaragoza,Alavés,tie
17861,2000-2001,1,3,Real Sociedad,Dep. La Coruña,tie
17862,2000-2001,1,3,UD Las Palmas,Real Valladolid,tie
...,...,...,...,...,...,...
21283,2009-2010,1,3,Athletic,Villarreal,home
21284,2009-2010,1,3,Real Zaragoza,Real Valladolid,visitor
21285,2009-2010,1,3,UD Almería,Getafe,home
21286,2009-2010,1,3,Real Madrid,Xerez CD,home


In [50]:
# Hacemos el merge del dataframe para train
df_train = pd.merge(df_games_train, df_teams_train, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_train = pd.merge(df_train, df_teams_train, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_train

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2000-2001,1,3,Barcelona,Racing,home,2009-2010,1,2.0,Barcelona,...,9.0,Racing,3,4.0,6.0,-2.0,1.0,1.0,1.0,4.0
1,2000-2001,1,3,Barcelona,Racing,home,2009-2010,1,2.0,Barcelona,...,17.0,Racing,3,2.0,4.0,-2.0,0.0,1.0,2.0,2.0
2,2000-2001,1,3,Barcelona,Racing,home,2009-2010,1,2.0,Barcelona,...,7.0,Racing,3,2.0,1.0,1.0,1.0,0.0,2.0,5.0
3,2000-2001,1,3,Barcelona,Racing,home,2009-2010,1,2.0,Barcelona,...,20.0,Racing,3,0.0,5.0,-5.0,0.0,3.0,0.0,0.0
4,2000-2001,1,3,Barcelona,Racing,home,2009-2010,1,2.0,Barcelona,...,16.0,Racing,3,1.0,2.0,-1.0,1.0,2.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6147,2009-2010,1,3,Valencia,Sporting Gijón,tie,2002-2003,1,3.0,Valencia,...,20.0,Sporting Gijón,3,5.0,12.0,-7.0,0.0,3.0,0.0,0.0
6148,2009-2010,1,3,Valencia,Sporting Gijón,tie,2001-2002,1,3.0,Valencia,...,10.0,Sporting Gijón,3,3.0,5.0,-2.0,1.0,1.0,1.0,4.0
6149,2009-2010,1,3,Valencia,Sporting Gijón,tie,2001-2002,1,3.0,Valencia,...,20.0,Sporting Gijón,3,5.0,12.0,-7.0,0.0,3.0,0.0,0.0
6150,2009-2010,1,3,Valencia,Sporting Gijón,tie,2000-2001,1,4.0,Valencia,...,10.0,Sporting Gijón,3,3.0,5.0,-2.0,1.0,1.0,1.0,4.0


In [51]:
#Esto es otra opción para seleccionar las temporadas a las que vamos a hacer el train
#Define el rango de temporadas como una cadena
#temporadas = "2000:2010"

#Divide el rango en años de inicio y final
#temporada_inicio, temporada_fin = temporadas.split(":")m
#temporada_inicio = int(temporada_inicio)
#temporada_fin = int(temporada_fin)

#Filtra las filas correspondientes al rango de temporadas
#df_entrenamiento = df[(df['season'] >= f"{temporada_inicio}-{temporada_inicio + 1}")
#                      & (df['season'] <= f"{temporada_fin - 1}-{temporada_fin}")]

In [52]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])

In [53]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [54]:
df_teams_test = df_teams[(df_teams["season"] == season) & (df_teams["division"] == division) & 
                         (df_teams["matchday"] == matchday)]
df_games_test = df_games[(df_games["season"] == season) & (df_games["division"] == division) &
                         (df_games["matchday"] == matchday)]

In [55]:
# Hacemos el merge del dataframe para hacer el test
df_test = pd.merge(df_games_test, df_teams_test, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_test = pd.merge(df_test, df_teams_test, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_test

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,3,Sevilla FC,Celta de Vigo,tie,2019-2020,1,3.0,Sevilla FC,...,11.0,Celta de Vigo,3,3.0,4.0,-1.0,1.0,1.0,1.0,4.0
1,2019-2020,1,3,Athletic,Real Sociedad,home,2019-2020,1,2.0,Athletic,...,13.0,Real Sociedad,3,2.0,3.0,-1.0,1.0,1.0,1.0,4.0
2,2019-2020,1,3,CA Osasuna,Barcelona,tie,2019-2020,1,6.0,CA Osasuna,...,8.0,Barcelona,3,7.0,5.0,2.0,1.0,1.0,1.0,4.0
3,2019-2020,1,3,Getafe,Alavés,tie,2019-2020,1,17.0,Getafe,...,7.0,Alavés,3,2.0,1.0,1.0,1.0,0.0,2.0,5.0
4,2019-2020,1,3,Levante,Real Valladolid,home,2019-2020,1,4.0,Levante,...,12.0,Real Valladolid,3,3.0,4.0,-1.0,1.0,1.0,1.0,4.0
5,2019-2020,1,3,Real Betis,CD Leganés,home,2019-2020,1,15.0,Real Betis,...,20.0,CD Leganés,3,1.0,4.0,-3.0,0.0,3.0,0.0,0.0
6,2019-2020,1,3,Valencia,RCD Mallorca,home,2019-2020,1,10.0,Valencia,...,14.0,RCD Mallorca,3,2.0,4.0,-2.0,1.0,2.0,0.0,3.0
7,2019-2020,1,3,Atlético Madrid,SD Eibar,home,2019-2020,1,1.0,Atlético Madrid,...,18.0,SD Eibar,3,3.0,5.0,-2.0,0.0,2.0,1.0,1.0
8,2019-2020,1,3,Espanyol,Granada CF,visitor,2019-2020,1,19.0,Espanyol,...,9.0,Granada CF,3,7.0,5.0,2.0,1.0,1.0,1.0,4.0
9,2019-2020,1,3,Villarreal,Real Madrid,tie,2019-2020,1,16.0,Villarreal,...,5.0,Real Madrid,3,6.0,4.0,2.0,1.0,0.0,2.0,5.0


In [56]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])

In [57]:
X_test = df_test[['rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [58]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [59]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [60]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.4


In [61]:
# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)']))

                precision    recall  f1-score   support

   home_win(1)       0.60      0.60      0.60         5
visitor_win(2)       0.50      0.25      0.33         4
        tie(X)       0.00      0.00      0.00         1

      accuracy                           0.40        10
     macro avg       0.37      0.28      0.31        10
  weighted avg       0.50      0.40      0.43        10

